In [1]:
from pymongo import MongoClient
import tushare
import json
import requests
from fake_useragent import UserAgent
import datetime

class IwencaiNorth:

    def __init__(self):
        self.collection=MongoClient()['wencai']['north']

    def get_trade_date(self):
#         tushare.set_token('8b8f77e8abc8396a88d99ee51fd76a602fbe8fb3c0eae86c6561e3d7')
#         pro = tushare.pro_api()
#         df = pro.trade_cal(exchange='', start_date='20200926', end_date='20201012', is_open='1')
#         date = df['cal_date'].to_list()
        day = datetime.datetime.now()
        date=['20201110']
#         date = [day.strftime('%Y%m%d')]

        return date

    def get_data(self, question):
        self.headers = {
            'Host': 'www.iwencai.com',
            'Origin': 'http://www.iwencai.com',
            'Referer': 'http://www.iwencai.com/unifiedwap/result?w=%s&querytype=&issugs' % question,
            'User-Agent': UserAgent(path='D://fake_useragent.json').random,
            'Accept - Encoding': 'gzip, deflate',
            'Accept - Language': 'zh - CN, zh;q = 0.9, ja;q = 0.8, ko;q = 0.7'
        }
        self.post_url = 'http://www.iwencai.com/unifiedwap/unified-wap/result/get-stock-pick'
        self.data_temp = {
            'question': question,
            'secondary_intent': 'stock',
            'perpage': 5000,
            'page': 1, }
        self.cookies = {'chat_bot_session_id': '469324f4d5b911bf9c05def3ee800dab',
                        ' PHPSESSID': 'b50a91cc70dc14f137a78b8f2db9282d',
                        ' cid': 'b50a91cc70dc14f137a78b8f2db9282d1582445631',
                        ' ComputerID': 'b50a91cc70dc14f137a78b8f2db9282d1582445631', ' guideState': '1',
                        ' other_uid': 'Ths_iwencai_Xuangu_087b043c9b7bf78a61804a7bd7b5b912',
                        ' user': 'MDptb18yMTEzMTY4NzI6Ok5vbmU6NTAwOjIyMTMxNjg3Mjo3LDExMTExMTExMTExLDQwOzQ0LDExLDQwOzYsMSw0MDs1LDEsNDA7MSwxLDQwOzIsMSw0MDszLDEsNDA7NSwxLDQwOzgsMDAwMDAwMDAwMDAwMDAwMDAwMDAwMDEsNDA6MjQ6OjoyMTEzMTY4NzI6MTU4Mjg3Mzc2Mjo6OjE0MjAxNjk1MjA6ODY0MDA6MDoxYjE0ZWNiNjg1ZmNkODMzZWFhNGFmOGZlYTc1NDE1NjI6ZGVmYXVsdF80OjE%3D',
                        ' userid': '211316872', ' u_name': 'mo_211316872', ' escapename': 'mo_211316872',
                        ' ticket': '595ce825965bcf62ce6b927820fe9aae',
                        ' v': 'Ap7K_6bJTJHijJj1t6SQFYZh6T_gX2La9CAW-EgnCiDdZzHlsO-y6cSzZsQb'}
        response = requests.post(self.post_url, data=self.data_temp, headers=self.headers, cookies=self.cookies)
        resp = response.content.decode(encoding="utf-8", errors="replace")
        return json.loads(resp)

    def save_to_mongo(self,data,i):
        self.collection.insert_one(data)
        print('{}数据已写入'.format(i))


    def run(self):
        trade_date=self.get_trade_date()
        for i in trade_date:
            i_format='%s年%s月%s日' % (i[0:4], i[4:6], i[6:8])
            question_str = i_format + '港资流向'
            question=question_str.encode('utf-8')
            datas=self.get_data(question)['data']['data']
            north_list=[]
            for data in datas:
                item={}
                item['股票简称']=data['股票简称']
                item['股票代码']=data['股票代码']
                item['陆股通持股量']=data['陆股通持股量[{}]'.format(i)]
                item['陆股通卖出金额']=data['陆股通卖出金额[{}]'.format(i)]
                item['陆股通成交金额']=data['陆股通成交金额[{}]'.format(i)]
                item['陆股通净买入额']=data['陆股通净买入额[{}]'.format(i)]
                item['陆股通持股占流通a股比']=data['陆股通持股占流通a股比[{}]'.format(i)]
                north_list.append(item)
            n_data={'date':i,'data':north_list}
            self.save_to_mongo(n_data,i)       

if __name__ == '__main__':
    north=IwencaiNorth()
    north.run()


20201110数据已写入
